In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [78]:
# get the urls of all items
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
def get_all_item_urls(driver, category_url):
    driver.get(category_url)
    time.sleep(3)  # wait for 3 seconds
    try:
        accept_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        accept_btn.click()
        print("Cookie button clicked")
    except Exception as e:
        print("Failed to click cookie button:", e)

    url_set = set()
    last_count = 0
    while True:
        product_divs = driver.find_elements(
            By.XPATH,
            '//div[@data-retailer-anchor="product-list"]//div[starts-with(@data-test, "fop-wrapper")]'
        )
        

        for div in product_divs:
            try:
                a_tag = div.find_element(By.XPATH, './/a[@data-test="fop-product-link"]')
                url_set.add(a_tag.get_attribute("href"))
            except:
                continue
        print(f"Current scraped {len(url_set)} products")

        if last_count == len(url_set):
            break
        
        last_count = len(url_set)
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(3)
    
    return list(url_set)


In [89]:
category_urls = {
    "meat": "https://handlaprivatkund.ica.se/stores/1004181/categories/k%C3%B6tt-chark-f%C3%A5gel/3ed155d4-01df-408f-bffc-f91e7404cab2?source=navigation",
    "own": "https://handlaprivatkund.ica.se/stores/1004181/categories/butikens-egna-varor/5dc3164f-038a-4202-a734-86a0bac7c427?source=navigation",
    "fish": "https://handlaprivatkund.ica.se/stores/1004181/categories/fisk-skaldjur/e32fbd57-fb20-4fdf-b1f6-f83cb990ccd8?source=navigation",
    "cheese": "https://handlaprivatkund.ica.se/stores/1004181/categories/mejeri-ost/e2ac37e6-aa68-477b-b78a-97228046b8d2?source=navigation",
    "bread": "https://handlaprivatkund.ica.se/stores/1004181/categories/br%C3%B6d-kakor/528b82cc-e3d4-43f2-9559-6e8ca9b56c2e?source=navigation",
    "vege": "https://handlaprivatkund.ica.se/stores/1004181/categories/vegetariskt/a931178f-85ab-4139-a54f-3dfe60421e36?source=navigation",
    "ready": "https://handlaprivatkund.ica.se/stores/1004181/categories/f%C3%A4rdigmat/ecfe2469-b003-4b3d-8e1d-9ef26d5c7328?source=navigation",
    "snack": "https://handlaprivatkund.ica.se/stores/1004181/categories/glass-godis-snacks/05f32791-8dc2-43cf-8230-1fcf0e05a9a9?source=navigation",
    "drink": "https://handlaprivatkund.ica.se/stores/1004181/categories/dryck/38b1e8f0-0146-4a5c-ba0c-286b8277d235?source=navigation",
    "pantry": "https://handlaprivatkund.ica.se/stores/1004181/categories/skafferi/7b81630c-9bad-4210-977d-a45aec0a5c9e?source=navigation",
    "frozen": "https://handlaprivatkund.ica.se/stores/1004181/categories/fryst/4aeec528-18cd-4b0f-9c50-21ac44e6f3e7?source=navigation",
}

In [90]:
for category in category_urls:
    driver = webdriver.Chrome()
    category_url = category_urls[category]
    url_lst = get_all_item_urls(driver, category_url)
    driver.quit()

    url_lst = pd.DataFrame(url_lst, columns=["url"])
    url_lst.to_csv(f"ica_{category}_urls.csv", index=False)


Cookie button clicked
Current scraped 10 products
Current scraped 14 products
Current scraped 23 products
Current scraped 33 products
Current scraped 41 products
Current scraped 50 products
Current scraped 60 products
Current scraped 70 products
Current scraped 75 products
Current scraped 85 products
Current scraped 95 products
Current scraped 105 products
Current scraped 115 products
Current scraped 125 products
Current scraped 130 products
Current scraped 140 products
Current scraped 150 products
Current scraped 157 products
Current scraped 157 products
Cookie button clicked
Current scraped 10 products
Current scraped 20 products
Current scraped 25 products
Current scraped 35 products
Current scraped 35 products
Cookie button clicked
Current scraped 10 products
Current scraped 20 products
Current scraped 24 products
Current scraped 34 products
Current scraped 44 products
Current scraped 54 products
Current scraped 64 products
Current scraped 64 products
Cookie button clicked
Current 

In [ ]:
def get_item_info(driver, url):
    driver.get(url)

    res_dict = {}

    time.sleep(3)  # wait for 3 seconds

    # get item names
    product_name = driver.find_element(By.TAG_NAME, "h1").text
    res_dict['Name'] = product_name

    # get price and size
    price_div = driver.find_element(By.CSS_SELECTOR, 'div[data-test="price-container"]')
    price_text = price_div.find_element(By.TAG_NAME, "span").text
    price_text = price_text.replace("\xa0", " ").strip()
    res_dict['Price'] = price_text

    size_div = driver.find_element(By.CSS_SELECTOR, 'div[data-test="size-container"]')
    size_text = size_div.find_element(By.TAG_NAME, "span").text
    size_text = size_text.replace("\xa0", " ").strip()
    res_dict['Size'] = size_text

    # get nutrition values
    try:
        tbody = driver.find_element(By.TAG_NAME, "tbody")
        tds = tbody.find_elements(By.TAG_NAME, "td")
        nutrition_values = [td.text.strip() for td in tds]

        for i in range(0,len(nutrition_values), 3):
            res_dict[nutrition_values[i]] = nutrition_values[i+1]
    except:
        pass

    return res_dict

# test
# driver = webdriver.Chrome()
# url = "https://handlaprivatkund.ica.se/stores/1004181/products/italiensk-lantbr%C3%B6d-os%C3%B6tat-650g-p%C3%A5gen/2063074"

# res = get_item_info(driver, url)
# print(res)

# driver.quit()


In [ ]:
import random
for category in category_urls:
    url_lst = pd.read_csv(f"ica_{category}_urls.csv")['url'].tolist()
    df = pd.DataFrame()

    driver = webdriver.Chrome()
    # url_lst = [
    #     "https://handlaprivatkund.ica.se/stores/1004181/products/finbladig-rucola-sk%C3%B6ljd-65g-ica/1396762",
    #     "https://handlaprivatkund.ica.se/stores/1004181/products/n%C3%B6tf%C3%A4rs-f%C3%A4rsk-12-1kg-ica/2117200",
    #     "https://handlaprivatkund.ica.se/stores/1004181/products/italiensk-lantbr%C3%B6d-os%C3%B6tat-650g-p%C3%A5gen/2063074",
    # ]

    
    for url in url_lst:
        res = get_item_info(driver, url)
        time.sleep(random.uniform(1, 3))  # random wait between 1 to 3 seconds
        
        new_row = pd.DataFrame([res])
        df = pd.concat([df, new_row], ignore_index=True)

    driver.quit()

    df.to_csv(f"ica_{category}_data.csv", index=False)

In [94]:
for category in category_urls:
    df = pd.read_csv(f"ica_{category}_data.csv")
    url_df = pd.read_csv(f"ica_{category}_urls.csv")
    df['url'] = url_df['url']
    df.to_csv(f"ica_{category}_data.csv", index=False)
